## Img2Prompt-VQA: Inference Demo

In [2]:
# install requirements
import sys
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !git clone https://github.com/salesforce/LAVIS
    %cd LAVIS
    !pip install .
    !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz
else:
    !pip install omegaconf
    %cd ../..
    !pip install .
    !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz


/Users/jiaxianguo
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
     |████████████████████████████████| 13.7 MB 5.8 MB/s eta 0:00:01
  Using cached spacy-3.0.8-cp36-cp36m-macosx_10_9_x86_64.whl (5.9 MB)
  Using cached cymem-2.0.7.tar.gz (9.9 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached spacy_legacy-3.0.10-py2.py3-none-any.whl (21 kB)
  Using cached thinc-8.0.17-cp36-cp36m-macosx_10_9_x86_64.whl (635 kB)
  Using cached preshed-3.0.8.tar.gz (14 kB)
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/jiaxianguo/anaconda3/bin/python /Users/jiaxianguo/anaconda3/lib/python3.6/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/cc/bz4g9_g16098fysc1jl9g4b80000gn/T/pip-build-env-aiarbz53/overlay --no-warn-script-location --

  Using cached preshed-3.0.7-cp36-cp36m-macosx_10_9_x86_64.whl (107 kB)
  Using cached wasabi-0.10.1-py3-none-any.whl (26 kB)
  Using cached blis-0.7.9.tar.gz (2.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached srsly-2.4.5.tar.gz (349 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached typer-0.3.2-py3-none-any.whl (21 kB)
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached pydantic-1.8.2-cp36-cp36m-macosx_10_9_x86_64.whl (2.6 MB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached murmurhash-1.0.9.tar.gz (12 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend depend

  ERROR: Command errored out with exit status 1:
   command: /Users/jiaxianguo/anaconda3/bin/python /Users/jiaxianguo/anaconda3/lib/python3.6/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /var/folders/cc/bz4g9_g16098fysc1jl9g4b80000gn/T/tmplokq9pjp
       cwd: /private/var/folders/cc/bz4g9_g16098fysc1jl9g4b80000gn/T/pip-install-pv9zbw4u/cymem_d82b3ef1959c4659b7a3285b61b98704
  Complete output (21 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.7-x86_64-3.6
  creating build/lib.macosx-10.7-x86_64-3.6/cymem
  copying cymem/__init__.py -> build/lib.macosx-10.7-x86_64-3.6/cymem
  copying cymem/about.py -> build/lib.macosx-10.7-x86_64-3.6/cymem
  creating build/lib.macosx-10.7-x86_64-3.6/cymem/tests
  copying cymem/tests/__init__.py -> build/lib.macosx-10.7-x86_64-3.6/cymem/tests
  copying cymem/tests/test_import.py -> build/lib.macosx-10.7-x86_64-3.6/cymem/tests
  copying cymem/cymem.pyx -> build/lib.

In [3]:
import torch
import requests
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

from lavis.common.gradcam import getAttMap
from lavis.models import load_model_and_preprocess

ModuleNotFoundError: No module named 'iopath'

### Load an example image and question

In [ ]:
#img_url = 'https://storage.googleapis.com/sfr-vision-language-research/LAVIS/projects/pnp-vqa/demo.png'
#raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_image = Image.open("./demo.png").convert("RGB")
#display(raw_image.resize((400, 300)))
question = "What item s are spinning which can be used to control electric?"
print(question)

In [ ]:
# setup device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### Load Img2Prompt-VQA model

In [ ]:
model, vis_processors, txt_processors = load_model_and_preprocess(name="img2prompt_vqa", model_type="base", is_eval=True, device=device)

### Preprocess image and text inputs

In [ ]:
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
question = txt_processors["eval"](question)

samples = {"image": image, "text_input": [question]}

### Img2Prompt-VQA utilizes 4 submodels to perform VQA:
#### 1. Image-Question Matching 
Compute the relevancy score of image patches with respect to the question using GradCAM

In [ ]:
samples = model.forward_itm(samples=samples)

In [ ]:
# Gradcam visualisation
dst_w = 720
w, h = raw_image.size
scaling_factor = dst_w / w

resized_img = raw_image.resize((int(w * scaling_factor), int(h * scaling_factor)))
norm_img = np.float32(resized_img) / 255
gradcam = samples['gradcams'].reshape(24,24)

avg_gradcam = getAttMap(norm_img, gradcam, blur=True)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(avg_gradcam)
ax.set_yticks([])
ax.set_xticks([])
print('Question: {}'.format(question))

#### 2. Image Captioning
Generate question-guided captions based on the relevancy score

In [ ]:
samples = model.forward_cap(samples=samples, num_captions=50, num_patches=20)
print('Examples of question-guided captions: ')
samples['captions'][0][:5]

#### 3. Question Generation
Generate synthetic questions using the captions

In [ ]:
samples = model.forward_qa_generation(samples)
print('Sample Question: {} \nSample Answer: {}'.format(samples['questions'][:5], samples['answers'][:5]))

#### 4. Prompt Construction
Prepare the prompts for LLM

In [ ]:
Img2Prompt = model.prompts_construction(samples)

#### 4. Load LLM and Predict Answers


In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
def load_model(model_selection):
    model = AutoModelForCausalLM.from_pretrained(model_selection)
    tokenizer = AutoTokenizer.from_pretrained(model_selection, use_fast=False)
    return model,tokenizer
def postprocess_Answer(text):
    for i, ans in enumerate(text):
        for j, w in enumerate(ans):
            if w == '.' or w == '\n':
                ans = ans[:j].lower()
                break
    return ans

model,tokenizer = load_model('facebook/opt-6.7b')

Img2Prompt_input = tokenizer(Img2Prompt, padding='longest', truncation=True, return_tensors="pt").to(
    device)

assert (len(Img2Prompt_input.input_ids[0])+20) <=2048
# print(len(question_input.attention_mask[0]))

outputs_list  = []
outputs = model.generate(input_ids=Img2Prompt_input.input_ids,
                         attention_mask=Img2Prompt_input.attention_mask,
                         max_length=20+len(Img2Prompt_input.input_ids[0]),
                         return_dict_in_generate=True,
                         output_scores = True
                         )
outputs_list.append(outputs)
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
def load_model(model_selection):
    model = AutoModelForCausalLM.from_pretrained(model_selection)
    tokenizer = AutoTokenizer.from_pretrained(model_selection, use_fast=False)
    return model,tokenizer

def postprocess_Answer(text):
    for i, ans in enumerate(text):
        for j, w in enumerate(ans):
            if w == '.' or w == '\n':
                ans = ans[:j].lower()
                break
    return ans

model,tokenizer = load_model('facebook/opt-6.7b')  # you can also use opt-13b/opt-30b/opt-66b, opt-175b if you have network weights

Img2Prompt_input = tokenizer(Img2Prompt, padding='longest', truncation=True, return_tensors="pt").to(
    device)

assert (len(Img2Prompt_input.input_ids[0])+20) <=2048
# print(len(question_input.attention_mask[0]))

outputs_list  = []
outputs = model.generate(input_ids=Img2Prompt_input.input_ids,
                         attention_mask=Img2Prompt_input.attention_mask,
                         max_length=20+len(Img2Prompt_input.input_ids[0]),
                         return_dict_in_generate=True,
                         output_scores = True
                         )
outputs_list.append(outputs)
